In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB   
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

C:\Users\matia\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data = pd.read_csv('clean_data.csv')

data_sample = data
# replace URL with 0 or 1
data_sample['URL'] = data_sample['URL'].replace(np.nan, 0)
data_sample['URL'][data_sample.URL != 0] = 1

# nr of URL and non-URL
print(len(data_sample['URL'][data_sample.URL == 1]),len(data_sample['URL'][data_sample.URL == 0]))

6614 49412


C:\Users\matia\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [3]:
pd.value_counts(data_sample['URL'])

0    49412
1     6614
Name: URL, dtype: int64

In [4]:
pd.value_counts(data_sample['Text'])

In a first in the nation, Oregon has passed a ballot measure decriminalizing the possession of heroin, oxycodone, meth and other hard drugs. Instead of possible jail time, a person would be able to pay a $100 fine or enter addiction treatment.\n                                                                                                                  993
“If Biden has more votes in Trump than Pennsylvania, Trump was never ‘ahead,’ Biden didn't ‘catch up.’ Biden is &amp; was always the winner and Trump is/was always the loser. Counting ballots isn't a baseball game. The time a ballot is counted doesn't give it some different valence”                                                                            654
Here is another 2010 Forbes article\n\n“The Pandemic That Wasn't”\n\n“The WHO widely criticized.. Parliamentary Assembly of Council of Europe announced plans to debate ‘false pandemics, a threat to health’.. WHO’s policy role should be drastically limited.”\n▶️             

In [5]:
data_sample[data_sample['Text']=='In a first in the nation, Oregon has passed a ballot measure decriminalizing the possession of heroin, oxycodone, meth and other hard drugs. Instead of possible jail time, a person would be able to pay a $100 fine or enter addiction treatment.\n']

,Unnamed: 0,Column1,ID,Date,Text,Domain,URL
52029,52029,52029,1.324450e+18,11/5/2020 20:18,"In a first in the nation, Oregon has passed a ...",unknown,0
52030,52030,52030,1.324450e+18,11/5/2020 20:18,"In a first in the nation, Oregon has passed a ...",unknown,0
52031,52031,52031,1.324450e+18,11/5/2020 20:17,"In a first in the nation, Oregon has passed a ...",unknown,0
52035,52035,52035,1.324450e+18,11/5/2020 20:16,"In a first in the nation, Oregon has passed a ...",unknown,0
52036,52036,52036,1.324440e+18,11/5/2020 20:09,"In a first in the nation, Oregon has passed a ...",unknown,0
...,...,...,...,...,...,...,...
53355,53355,53355,1.324120e+18,11/4/2020 22:36,"In a first in the nation, Oregon has passed a ...",unknown,0
53356,53356,53356,1.324120e+18,11/4/2020 22:36,"In a first in the nation, Oregon has passed a ...",unknown,0
53357,53357,53357,1.324120e+18,11/4/2020 22:36,"In a first in the nation, Oregon has passed a ...",unknown,0
53358,53358,53358,1.324120e+18,11/4/2020 22:36,"In a first in the nation, Oregon has passed a ...",unknown,0


In [6]:
len(data_sample['Text'].unique())

20952

In [7]:
data_new=pd.DataFrame(columns=data_sample.columns)
data_new

,Unnamed: 0,Column1,ID,Date,Text,Domain,URL


In [8]:
data_new=pd.DataFrame()
Texts=data_sample['Text'].unique()
for text in Texts:
    data_new=pd.concat([data_new, data_sample[data_sample['Text']==text].head(1)])

In [9]:
data_new=data_new.reset_index()
data_new=data_new.drop(columns=['Unnamed: 0','Column1'])

In [10]:
pd.value_counts(data_new['URL'])

0    19057
1     1895
Name: URL, dtype: int64

In [11]:
data_new.to_csv("Problem1_processed.csv",encoding='utf-8-sig')

In [12]:
data_1s=data_new[data_new['URL']==1]
data_0s=data_new[data_new['URL']==0]

In [13]:
data_0s_sample=data_0s.sample(n = 1900) 

In [14]:
data_balance=pd.concat([data_1s, data_0s_sample])


In [15]:
data_balance

,index,ID,Date,Text,Domain,URL
1,1,1.324130e+18,11/4/2020 23:25,“For Premier Doug Ford to say that they've don...,Medical,1
8,8,1.324130e+18,11/4/2020 23:24,"In any season, getting sick is tough. But rig...",Medical,1
12,13,1.324130e+18,11/4/2020 23:23,@FatEmperor And the lastest vicim of the swine...,Medical,1
17,18,1.324130e+18,11/4/2020 23:22,Fargo Cass Public Health holds additional flu ...,Medical,1
19,21,1.324130e+18,11/4/2020 23:32,Health over Illness - #COVID19\nAsk The Expert...,Medical,1
...,...,...,...,...,...,...
721,4280,1.324080e+18,11/4/2020 20:14,@nancygedge How is your son generally health w...,Medical,0
1377,7545,1.324040e+18,11/4/2020 17:13,Getting a flu vaccine this fall can reduce you...,Medical,0
978,4816,1.324060e+18,11/4/2020 19:01,"I scream, you scream, we all scream for the va...",Medical,0
18235,48849,1.324400e+18,11/5/2020 17:20,Volk Digital Clear Mag for Optic Disc &amp; Po...,unknown,0


In [16]:
data_balance.to_csv("Problem1_balance.csv",encoding='utf-8-sig')